In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs
import json
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Load data
foods = pd.read_csv('Food.csv')
drinks = pd.read_csv('Beverage.csv')

# Combine data
data = pd.concat([foods, drinks])

# Preprocess data
def preprocess_amount(amount):
    if isinstance(amount, str):
        return float(amount.split('/')[0].replace('g', '').replace('mL', ''))
    else:
        return amount

data['Amount for Diet (g/mL)'] = data['Amount for Diet (g/mL)'].apply(preprocess_amount)
data['Amount for Bulking (g/mL)'] = data['Amount for Bulking (g/mL)'].apply(preprocess_amount)

# Create feature for diet or bulking
data['Is_Diet'] = data['Amount for Diet (g/mL)'].notnull().astype(int)
data['Is_Bulking'] = data['Amount for Bulking (g/mL)'].notnull().astype(int)

# Input features and target
X = data[['Is_Diet', 'Is_Bulking', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)']]
y = data['Category']

# Encode labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_encoded = to_categorical(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build model
def build_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Compile model
model = build_model(X_train.shape[1:], y_encoded.shape[1])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Save LabelEncoder as JSON
#label_encoder_json = label_encoder.classes_.tolist()
#with open('label_encoder.json', 'w') as f:
#    json.dump(label_encoder_json, f)

# Save LabelEncoder as Pickle
joblib.dump(label_encoder, 'label_encoder.pkl')

# Save model
model.save('food_recommendation_model.h5')

# Convert model to TensorFlow.js
#tfjs.converters.save_keras_model(model, 'tfjs_model')


Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.4957 - loss: 8.2286 - val_accuracy: 0.4740 - val_loss: 1.6767
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5052 - loss: 3.1826 - val_accuracy: 0.4760 - val_loss: 1.4585
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4934 - loss: 2.2970 - val_accuracy: 0.5013 - val_loss: 1.1841
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5146 - loss: 1.8236 - val_accuracy: 0.4740 - val_loss: 1.1736
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5059 - loss: 1.6676 - val_accuracy: 0.4960 - val_loss: 1.1447
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5144 - loss: 1.4903 - val_accuracy: 0.4833 - val_loss: 1.1349
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4897 - loss: 1.3837 - val_accuracy: 0.5247 - val_loss: 1.1164
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4907 - loss: 1.3362 - val_accuracy: 0.4980 - val_los

In [ ]:
import pandas as pd

# Load datasets
food_data = pd.read_csv('Food.csv')
beverage_data = pd.read_csv('Beverage.csv')

# Define function for filtering food
def filter_food(height, weight, desired_weight, is_diet, num_choices=50):
    # Calculate the caloric needs
    if is_diet:
        calories_needed = (weight - desired_weight) * 7700 / 30  # Simple weight loss estimation
    else:
        calories_needed = (desired_weight - weight) * 7700 / 30  # Simple weight gain estimation

    # Filter food data
    if is_diet:
        filtered_food = food_data[(food_data['Category'] == 'Diet') & (food_data['Calories (kcal)'] <= calories_needed)].sample(min(num_choices, len(food_data)))
    else:
        filtered_food = food_data[(food_data['Category'] == 'Bulking') & (food_data['Calories (kcal)'] <= calories_needed)].sample(min(num_choices, len(food_data)))

    return filtered_food

# Define function for filtering beverages
def filter_beverages(height, weight, desired_weight, is_diet, num_choices=50):
    # Calculate the caloric needs
    if is_diet:
        calories_needed = (weight - desired_weight) * 7700 / 30  # Simple weight loss estimation
    else:
        calories_needed = (desired_weight - weight) * 7700 / 30  # Simple weight gain estimation

    # Filter beverage data
    if is_diet:
        filtered_beverages = beverage_data[(beverage_data['Category'] == 'Diet') & (beverage_data['Calories (kcal)'] <= calories_needed)].sample(min(num_choices, len(beverage_data)))
    else:
        filtered_beverages = beverage_data[(beverage_data['Category'] == 'Bulking') & (beverage_data['Calories (kcal)'] <= calories_needed)].sample(min(num_choices, len(beverage_data)))

    return filtered_beverages

# Test the filtering functions
filtered_food = filter_food(170, 69, 80, is_diet=False)
filtered_drinks = filter_beverages(170, 69, 80, is_diet=False)

# Print Food Recommendations
print("Food Recommendations:")
for idx, row in filtered_food.iterrows():
    print(f"- Food Name: {row['Food Name']}")
    print(f"  Ingredients: {row['Ingredients']}")
    print(f"  Calories (kcal): {row['Calories (kcal)']}")
    print(f"  Protein (g): {row['Protein (g)']}")
    print(f"  Carbohydrates (g): {row['Carbohydrates (g)']}")
    print(f"  Fats (g): {row['Fats (g)']}\n")

# Print Drink Recommendations
print("\nDrink Recommendations:")
for idx, row in filtered_drinks.iterrows():
    print(f"- Beverage Name: {row['Beverage Name']}")
    print(f"  Ingredients: {row['Ingredients']}")
    print(f"  Calories (kcal): {row['Calories (kcal)']}")
    print(f"  Protein (g): {row['Protein (g)']}")
    print(f"  Carbohydrates (g): {row['Carbohydrates (g)']}")
    print(f"  Fats (g): {row['Fats (g)']}\n")


Food Recommendations:
- Food Name: Fried Rice
  Ingredients: Tomato 93g, Spices Cayenne Pepper
  Calories (kcal): 340
  Protein (g): 24
  Carbohydrates (g): 23
  Fats (g): 27

- Food Name: Garlic Bread
  Ingredients: Almond 58g, Spices Paprika
  Calories (kcal): 431
  Protein (g): 8
  Carbohydrates (g): 76
  Fats (g): 7

- Food Name: Vegetable Curry
  Ingredients: Onion 126g, Spices Cardamom, Garlic Powder, Black Pepper
  Calories (kcal): 424
  Protein (g): 44
  Carbohydrates (g): 29
  Fats (g): 14

- Food Name: Spicy Tacos
  Ingredients: Honey 71g, Spices Paprika, Garlic Powder
  Calories (kcal): 620
  Protein (g): 48
  Carbohydrates (g): 45
  Fats (g): 10

- Food Name: Jalapeno Poppers
  Ingredients: Salmon 136g, Spices Black Mustard Seed
  Calories (kcal): 670
  Protein (g): 16
  Carbohydrates (g): 57
  Fats (g): 10

- Food Name: Roasted Carrots
  Ingredients: Wheat Flour 87g, Spices Paprika
  Calories (kcal): 315
  Protein (g): 21
  Carbohydrates (g): 65
  Fats (g): 22

- Food Name